In [1]:
import json

import joblib
from pandas.core.interchange.dataframe_protocol import DataFrame

In [190]:
def get_temp_chill():
    temp = 2
    return temp
def get_temp_boiler():
    temp = 2
    return temp
def GPXX_check():
    temp = 2
    return temp
def room_check():
    temp = 2
    return temp
def get_occupation():
    temp = 2
    return temp
def get_elec_consumption():
    temp = 2
    return temp
def get_gaz_consumption():
    temp = 2
    return temp

In [174]:
class node():
    def __init__(self,type,text,options,on_action=False):
        self.type=type
        self.text=text
        self.options =options
        self.on_action=on_action
    def next(self,model_output,clf,tree):
        next_ = []
        labels = [opt["next"] for opt in main_node.options]
        print(clf.classes_)
        print(labels)
        for i in range(len(clf.classes_)):
            if clf.classes_[i] in labels:
                next_.append(model_output[i])
        print(next_)
        pred_class = labels[next_.index(max(next_))]
        confidence = max(next_)
        print(confidence)
        threshold = 1/len(clf.classes_)
        print(threshold)
        if confidence<threshold:
            self.type="handoff"
            self.text="problem due to bad comprehension you can either repeat or call that number :"
            self.options = self.options.copy()
            self.on_action=False
        if confidence>threshold:
            for i in tree["nodes"].keys():
                if i == pred_class:
                    self.type=tree["nodes"][i]["type"]
                    self.text= tree["nodes"][i]["text"]
                    self.options = tree["nodes"][i]["options"]
                    self.on_action=tree["nodes"][i]["on_action"]

In [8]:
import pandas as pd
file_path = "./dataset_LLM.xlsx"
df = pd.read_excel(file_path)

print(df)

                                          text            intent
0                              je veux réparer            fixing
1                 j ai besoin d une réparation            fixing
2                       service de maintenance            fixing
3                   un technicien pour réparer            fixing
4          besoin d une intervention technique            fixing
..                                         ...               ...
685                              sonde CHWS HS  chill_production
686                              sonde CHWR HS  chill_production
687  séquence de démarrage production froid KO  chill_production
688           priorité des chillers incorrecte  chill_production
689            alternance des chillers bloquée  chill_production

[690 rows x 2 columns]


# TFIDF

In [176]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Création du vectoriseur TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=1)

# Transformation du texte
X = vectorizer.fit_transform(df["text"])
y = df["intent"]
print("Shape des vecteurs :", X.shape)  # nb_exemples x nb_features
print("Exemple de features :", vectorizer.get_feature_names_out()[:10])


Shape des vecteurs : (690, 2632)
Exemple de features : ['101' '101 maintenant' '101 temperature' '101 valeur'
 '101 valeur actuelle' '1er' '1er étage' '203' '203 24h' '24h']


In [177]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

# Création et entraînement du modèle
clf = LogisticRegression(max_iter=2000)
clf.fit(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
joblib.dump(clf, "modele_LLM_tfidf.pkl")
print(classification_report(y_test, y_pred,zero_division=0))
print("Classes :", clf.classes_)

                      precision    recall  f1-score   support

          Hotel_info       0.54      1.00      0.70        21
             acceuil       1.00      0.11      0.20         9
       boiler_fixing       0.54      0.83      0.66        23
    chill_production       0.75      0.46      0.57        13
      chiller_fixing       0.65      0.79      0.71        19
        cogen_fixing       0.51      0.91      0.66        23
              fixing       0.00      0.00      0.00         9
     get_consumption       0.80      0.44      0.57         9
get_elec_consumption       0.67      0.22      0.33         9
 get_gaz_consumption       1.00      1.00      1.00         9
      get_occupation       1.00      0.33      0.50         9
     heat_production       1.00      0.11      0.20         9
               rooms       0.76      0.89      0.82        18
         temp_boiler       1.00      0.67      0.80         9
          temp_chill       1.00      0.56      0.71         9
       

In [178]:
def prediction(input):
    X_new = vectorizer.transform(input)
    proba = clf.predict_proba(X_new)[0]
    return proba

test = ["comment va la ventilation ?"]
proba = prediction(test)
print(proba)
pred_class = clf.classes_[proba.argmax()]
confidence = proba.max()
print(f"Intent prédit : {pred_class} (confiance {confidence:.2f})")



[0.04430576 0.02676999 0.0536924  0.04117054 0.03909048 0.05904285
 0.0255774  0.02062695 0.0250736  0.02122393 0.02621931 0.02876818
 0.06044408 0.0227571  0.0227357  0.48250174]
Intent prédit : ventilation (confiance 0.48)


# transformer

In [179]:
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
X_emb = model.encode(df["text"], convert_to_numpy=True, normalize_embeddings=True)
le = LabelEncoder()
y = le.fit_transform(df["intent"])


C:\Users\hugom\OneDrive\Documents\Git_Stage_2025_ZMH\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [180]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity

# Création et entraînement du modèle
clf_T = LogisticRegression(max_iter=2000)
clf_T.fit(X_emb, y)


X_train, X_test, y_train, y_test = train_test_split(X_emb, y, stratify=y, test_size=0.3, random_state=42)
clf_T.fit(X_train, y_train)
y_pred = clf_T.predict(X_test)
joblib.dump(clf_T, "modele_LLM_T.pkl")

print(classification_report(y_test, y_pred,zero_division=0))
print("Classes :", le.classes_)

              precision    recall  f1-score   support

           0       0.87      0.95      0.91        21
           1       0.89      0.89      0.89         9
           2       0.66      0.83      0.73        23
           3       0.80      0.62      0.70        13
           4       0.74      0.74      0.74        19
           5       0.61      0.83      0.70        23
           6       0.71      0.56      0.62         9
           7       0.80      0.89      0.84         9
           8       0.86      0.67      0.75         9
           9       1.00      1.00      1.00         9
          10       1.00      0.56      0.71         9
          11       1.00      0.22      0.36         9
          12       0.79      0.83      0.81        18
          13       0.78      0.78      0.78         9
          14       0.90      1.00      0.95         9
          15       1.00      0.89      0.94         9

    accuracy                           0.78       207
   macro avg       0.84   

In [182]:
print(le.classes_)

['Hotel_info' 'acceuil' 'boiler_fixing' 'chill_production'
 'chiller_fixing' 'cogen_fixing' 'fixing' 'get_consumption'
 'get_elec_consumption' 'get_gaz_consumption' 'get_occupation'
 'heat_production' 'rooms' 'temp_boiler' 'temp_chill' 'ventilation']


In [183]:
def prediction(input,model):
    X_emb = model.encode(input, convert_to_numpy=True, normalize_embeddings=True)
    proba = clf_T.predict_proba(X_emb)[0]
    return proba

test = ["comment va la ventilation ?"]
proba = prediction(test,model)
print(proba)

pred_class = le.classes_[proba.argmax()]
confidence = proba.max()
print(f"Intent prédit : {pred_class} (confiance {confidence:.2f})")



[0.0728931  0.02361585 0.12350931 0.03167968 0.05269084 0.02546408
 0.01640139 0.01621946 0.01848779 0.02972781 0.0440317  0.04343483
 0.05806254 0.03876899 0.03799468 0.36701796]
Intent prédit : ventilation (confiance 0.37)


C:\Users\hugom\OneDrive\Documents\Git_Stage_2025_ZMH\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


# TFIDF + transformers fréquence +semantique

In [184]:
import numpy as np
def prediction_T(input,model):
    X_emb = model.encode(input, convert_to_numpy=True, normalize_embeddings=True)
    proba = clf_T.predict_proba(X_emb)[0]
    return proba
def prediction_tfidf(input):
    X_new = vectorizer.transform(input)
    proba = clf.predict_proba(X_new)[0]
    return proba

def prediction_tfidf_T(input,model):
    proba_T = prediction_T(input,model)
    proba_tfidf = prediction_tfidf(input)
    proba = np.zeros(len(proba_T))
    for i in range(len(proba_T)):
        proba[i]=np.mean([proba_T[i],proba_tfidf[i]])
    return proba
test = ["comment va la ventilation ?"]
proba = prediction_tfidf_T(test,model)
print(proba)

pred_class = le.classes_[proba.argmax()]
confidence = proba.max()
print(f"Intent prédit : {pred_class} (confiance {confidence:.2f})")


[0.05859943 0.02519292 0.08860086 0.03642511 0.04589066 0.04225347
 0.0209894  0.0184232  0.02178069 0.02547587 0.0351255  0.03610151
 0.05925331 0.03076304 0.03036519 0.42475985]
Intent prédit : ventilation (confiance 0.42)


C:\Users\hugom\OneDrive\Documents\Git_Stage_2025_ZMH\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


# Embedding + SVC

In [10]:
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
X_emb = model.encode(df["text"], convert_to_numpy=True, normalize_embeddings=True)
le = LabelEncoder()
y = le.fit_transform(df["intent"])


C:\Users\hugom\OneDrive\Documents\Git_Stage_2025_ZMH\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity


svm = LinearSVC(C=1.0)
svc_cal = CalibratedClassifierCV(svm, cv=5)  # donne predict_proba
X_train, X_test, y_train, y_test = train_test_split(X_emb, y, stratify=y, test_size=0.3, random_state=42)
svc_cal.fit(X_train, y_train)
y_pred = svc_cal.predict(X_test)
joblib.dump(svc_cal, "modele_LLM_SVC.pkl")

print(classification_report(y_test, y_pred,zero_division=0))
print("Classes :", le.classes_)

              precision    recall  f1-score   support

           0       0.83      0.90      0.86        21
           1       0.89      0.89      0.89         9
           2       0.80      0.87      0.83        23
           3       0.69      0.69      0.69        13
           4       0.68      0.68      0.68        19
           5       0.77      0.74      0.76        23
           6       0.73      0.89      0.80         9
           7       0.90      1.00      0.95         9
           8       1.00      0.89      0.94         9
           9       1.00      1.00      1.00         9
          10       1.00      0.56      0.71         9
          11       0.67      0.44      0.53         9
          12       0.79      0.83      0.81        18
          13       0.80      0.89      0.84         9
          14       1.00      1.00      1.00         9
          15       1.00      1.00      1.00         9

    accuracy                           0.82       207
   macro avg       0.85   

In [188]:
def prediction_SVC(input,model):
    X_emb = model.encode(input, convert_to_numpy=True, normalize_embeddings=True)
    proba = svc_cal.predict_proba(X_emb)[0]
    return proba
test = ["comment va la ventilation ?"]
proba = prediction_tfidf_T(test,model)
print(proba)

pred_class = le.classes_[proba.argmax()]
confidence = proba.max()
print(f"Intent prédit : {pred_class} (confiance {confidence:.2f})")

[0.05859943 0.02519292 0.08860086 0.03642511 0.04589066 0.04225347
 0.0209894  0.0184232  0.02178069 0.02547587 0.0351255  0.03610151
 0.05925331 0.03076304 0.03036519 0.42475985]
Intent prédit : ventilation (confiance 0.42)


C:\Users\hugom\OneDrive\Documents\Git_Stage_2025_ZMH\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
tree_json = {
        "start": "acceuil",
        "nodes": {
            "acceuil": {"type": "message",
                        "text": """### welcome to the client service, please choose an option:
                                    * infos sur l'hotel
                                    * fixing """,
                        "options": [{"label": "Hotel info", "next": "Hotel_info"},
                                    {"label": "fixing", "next": "fixing"}],
                        "on_action": False},
            "Hotel_info": {"type": "message",
                           "text": """welcom to the Informations client service""",
                           "options": [{"label": "home", "next": "acceuil"},
                                       {"label": "consumption", "next": "get_consumption"},
                                       {"label": "occupation", "next": "get_occupation"},
                                       {"label": "rooms", "next": "rooms"},
                                       {"label": "temperature at the ouput off the boiler", "next": "temp_boiler"},
                                       {"label": "temperature at the ouput off the chiller", "next": "temp_chill"},
                                       {"label": "ventilation", "next": "ventilation"}],
                           "on_action": False},
            "fixing": {"type": "message",
                       "text": """welcom to the fixing client service""",
                       "options": [{"label": "home", "next": "acceuil"},
                                   {"label": "chill production", "next": "chill_production"},
                                   {"label": "heat production", "next": "heat_production"}],
                       "on_action": False},
            "ventilation": {"type": "action",
                            "text": "welcom to the ventilation client service",
                            "options": [],
                            "on_action": GPXX_check},
            "rooms": {"type": "form",
                      "text": "welcom to the room client service, please enter the room",
                      "options": [],
                      "on_action": room_check},

            "temp_chill": {"type": "action",
                           "text": False,
                           "options": [],
                           "on_action": get_temp_chill
                           },
            "temp_boiler": {"type": "action",
                            "text": False,
                            "options": [],
                            "on_action": get_temp_boiler
                            },
            "get_consumption": {"type": "message",
                                "text": "welcom to the consumption client service",
                                "options": [{"label": "home", "next": "acceuil"},
                                            {"label": "elec consumption", "next": "get_elec_consumption"},
                                            {"label": "gaz consumption", "next": "get_gaz_consumption"}],
                                "on_action": False},
            "get_elec_consumption": {"type": "action",
                                     "text": "welcom to the elec consumption client service",
                                     "options": [],
                                     "on_action": get_elec_consumption},
            "get_gaz_consumption": {"type": "action",
                                    "text": "welcom to the gaz consumption client service",
                                    "options": [],
                                    "on_action": get_gaz_consumption},
            "get_occupation": {"type": "action",
                               "text": "welcom to the occupation client service",
                               "options": [],
                               "on_action": get_occupation},
            "heat_production": {"type": "message",
                                "text": "welcom to the heat production client service",
                                "options": [{"label": "home", "next": "acceuil"},
                                            {"label": "boiler problem", "next": "boiler_fixing"},
                                            {"label": "cogen problem", "next": "cogen_fixing"}],
                                "on_action": False},
            "boiler_fixing": {"type": "message",
                              "text": """welcom to the boiler client service

                      •	Température de départ est trop basse : regarder le tableau des automates (disjoncteurs ou thermique),
                       vérifier la pression, vérifier la distribution pour s’assurer qu’il y a du débit (mais si T basse -> distribution fonctionne normalement),
                       aller voir sur place, vérifier le gaz avec la société de maintenance


                        o	Solution : forcer temporairement une ou plusieurs chaudières en manuel et sur la GTC
    """,
                              "options": [{"label": "home", "next": "acceuil"}],
                              "on_action": False},
            "cogen_fixing": {"type": "message",
                             "text": """welcom to the cogen client service
                      Pas assez de demande  les ballons sont trop chauds  elle ne démarre plus

    """,
                             "options": [{"label": "home", "next": "acceuil"}],
                             "on_action": False},
            "chill_production": {"type": "message",
                                 "text": "welcom to the chill production client service",
                                 "options": [{"label": "home", "next": "acceuil"},
                                             {"label": "chiller problem", "next": "chiller_fixing"}],
                                 "on_action": False},
            "chiller_fixing": {"type": "message",
                               "text": """welcom to the chiller client service
                     Les tours n’ont pas assez refroidi (message : pression trop haute)


                     Essayer de maintenir 28°C

    """,
                               "options": [{"label": "home", "next": "acceuil"}],
                               "on_action": False},

        }
    }


In [12]:
m = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")


In [13]:
m.save()